In [17]:
import hashlib

def hash_leaf(leaf: str) -> str:
    # return f'h({leaf})'
    return hashlib.sha256(leaf.encode()).hexdigest()

def hash_pair(left: str, right: str) -> str:
    # return f'h({left}, {right})'
    return hashlib.sha256((left + right).encode()).hexdigest()

def cut(s: str) -> str:
    return s[:5] + "..."

def calc_root(leaves: list[str]) -> list[list[str]]:
    tree = [[hash_leaf(l) for l in leaves]]
    tree[0].sort()

    n = len(tree[0])

    while n > 1:
        tree.append([])
        for i in range(0, n, 2):
            left = tree[-2][i]
            right = tree[-2][min(i + 1, n - 1)]
            if left > right:
                left, right = right, left
            tree[-1].append(hash_pair(left, right))
        n = (n + (n % 2)) // 2

    tree.reverse()

    return tree

def get_proof():
    pass

def verify():
    pass

leaves = ["A", "B", "C", "D", "E", "F", "G"]
hash_leaves = [hash_leaf(l) for l in leaves]
hash_leaves.sort()

indexes = [hash_leaves.index(hash_leaf(l)) for l in leaves]
print(indexes)

calc_root(leaves)

[2, 5, 3, 1, 4, 6, 0]


[['2359508bfcc5ea80358681e7e6e3bef69a42f89b7c9ace3d03afd8a0a8e91d93'],
 ['46bd38e72b1b95c2de6a8d892c2bad280ebaa9e1bb49a762d1911d78305675ae',
  'a0fbead813b245f415da5101342261b838a8cdb3e84cfe0ce2dc0a5970cf58fa'],
 ['38f403995062984c77a6fd8614d570a92fd75756f4a9b74548da52342efff22c',
  '19987eabf525296722a0657aa2938229006cb897c7efbc7659a6d9073d49e614',
  '97aa302b07ecd3b167757f4abc263dcd6dc53202b0b1428bfac81af0f53b7599',
  '169e848b6f7b622734fbc1b43f83eab9f066f4bf7b5dbdb202d5d42f2a3cd1ca'],
 ['333e0a1e27815d0ceee55c473fe3dc93d56c63e3bee2b3b4aee8eed6d70191a3',
  '3f39d5c348e5b79d06e842c114e6cc571583bbf44e4b0ebfda1a01ec05745d43',
  '559aead08264d5795d3909718cdd05abd49572e84fe55590eef31a88a08fdffd',
  '6b23c0d5f35d1b11f9b683f0b0a617355deb11277d91ae091d399c655b87940d',
  'a9f51566bd6705f7ea6ad54bb9deb449f795582d6529a0e22207b8981233ec58',
  'df7e70e5021544f4834bbee64a9e3789febc4be81470df629cad6ddb03320a5c',
  'f67ab10ad4e4c53121b6a5fe4da9c10ddee905b978d3788d2723d7bfacbe28a9']]